# Homework 8-2

---

## Imports

In [10]:
from thermostate import State, Q_, units, SystemInternational as SI

---

## Definitions

In [11]:
sub_v = 'water'
sub_a = 'air'

T_1 = Q_(28.0, 'degC')
p_1 = Q_(1.0, 'bar')
phi_1 = Q_(0.7, 'dimensionless')

Qdot_out = Q_(11.0, 'refrigeration_tons')

phi_2 = Q_(1.0, 'dimensionless')

T_3 = Q_(24.0, 'degC')
p_3 = Q_(1.0, 'bar')
phi_3 = Q_(0.4, 'dimensionless')

---

## Problem Statement

An air conditioner operating at steady state takes in moist air at 28.0 celsius, 1.0 bar, and 70.00%  relative humidity. The moist air first passes over a cooling coil in the dehumidifier unit and some water vapor is condensed. The rate of heat transfer between the moist air and the cooling coil is 11.0 ton_of_refrigeration. Saturated moist air and condensate streams exit the dehumidifier at the same temperature. The condensed water is discarded, while the remiaining moist air is directed to the heating unit where it is heated to 24.0 celsius, 1.0 bar and 40.00%  relative humidity. Neglecting kinetic and potential energy effects, determine

1. the temperature of the moist air and condensate exiting the dehumidifier, in degC
2. the volumetric flow rate of the air entering the air conditioner at state 1, in m3/min
3. the rate water is condensed and removed from the dehumidifier, in kg/min
4. the rate of heat transfer to the air passing through the heating unit, in kW

![moran_8e_fig_12_11.jpg](attachment:moran_8e_fig_12_11.jpg)

---

## Solution

### 1. the temperature of the moist air and condensate at 2

To find the outlet temperature of the dehumidifier section, we can draw a control volume around just that section of the device. The process from 1 to 2 is a variable composition cooling process, so we need to account for that in the mass balance for the water and in the energy rate balance. We will also need a control volume around the heating section of the device. The process from 2 to 3 is a constant composition heating process. Therefore, $\omega_2 = \omega_3$, and since the total pressure is constant, $p_{v2} = p_{v3}$. We further know that the mixture is saturated at state 2 ($\phi_2 =$ 100%), such that $p_{v2} = p_{g}(T_2)$. Therefore, we can find the saturation state where $p_{g}$ and $x$ are the independent variables.

In [12]:
st_g3 = State(sub_v, T=T_3, x=Q_(1.0, 'dimensionless'))
p_g3 = st_g3.p.to(SI.p) 

p_v3 = phi_3*p_g3

p_v2 = p_v3
st_v2 = State(sub_v, p=p_v2, x=Q_(1.0, 'dimensionless'))
T_2 = st_v2.T.to(SI.T)

The saturation pressure at state 3 is $p_{g3} =$ 0.0299 bar, and the actual partial pressure of water vapor is $p_{v3} = p_{v2} =$ 0.0119 bar

<div class="alert alert-success">

**Answer:** The temperature of the air exiting the dehumidifier is $T_2 =$ 9.58 celsius.

</div>

### 2. the volumetric flow rate of the air entering the inlet

To find the volumetric flow rate of the moist air, we can find the mass flow rate of the dry air and multiply by the specific volume of the dry air to get the volumetric flow rate. The mass flow rate of air is found by an energy balance around the dehumidifier, since we know the heat transfer rate. We can use the relative humidity at the inlet to find the partial pressure of water vapor there, and then the humidity ratio. The humidity ratio at the oulet (state 2) is found by the previously found partial pressure of water vapor.

In [13]:
MW_v = Q_(18.02, 'kg/kmol')
MW_a = Q_(28.97, 'kg/kmol')
p_2 = p_1
omega_3 = omega_2 = (MW_v/MW_a*p_v2/(p_2 - p_v2)).to('dimensionless')

st_g1 = State(sub_v, T=T_1, x=Q_(1.0, 'dimensionless'))
p_g1 = st_g1.p.to(SI.p)
p_v1 = phi_1*p_g1
omega_1 = (MW_v/MW_a*p_v1/(p_1 - p_v1)).to('dimensionless')

The saturation pressure at the inlet is $p_{g1} =$ 0.0378 bar, the partial pressure of water vapor there is $p_{v1} =$ 0.0265 bar, and the inlet humidity ratio is $\omega_1 =$ 0.0169 . The outlet humidity ratio is $\omega_3 =$ 0.00752 .

Now, the energy balance for the condenser is as follows
$$0 = \dot{Q}_{cv} + \dot{m}_a \left(h_{a1} - h_{a2}\right) + \dot{m}_{v1} h_{v1} - \dot{m}_{v2} h_{v2} - \dot{m}_{liq} h_{liq}$$

The mass rate balance for the water in the dehumidifier reads
$$\dot{m}_{liq} = \dot{m}_{v1} - \dot{m}_{v2}$$
and dividing through by the mass flow rate of the air
$$\frac{\dot{m}_{liq}}{\dot{m}_a} = \omega_1 - \omega_2$$

The overall energy balance can then be simplified to
$$\dot{m}_a = \frac{\dot{Q}_{cv}}{h_{a2} - h_{a1} + \omega_2 h_{v2} - \omega_1 h_{v1} - \left(\omega_2 - \omega_1\right) h_{liq}}$$

The enthalpies for the air are calculated using the partial pressure of the air and the temperature, while the enthalpies of the water vapor are found by (1) the partial pressure of the vapor and the temperature and (2) the saturated condition at the partial pressure of water vapor for the vapor and liquid.

In [15]:
p_a1 = p_1 - p_v1
st_a1 = State(sub_a, T=T_1, p=p_a1)
h_a1 = st_a1.h.to(SI.h)

p_a2 = p_2 - p_v2
st_a2 = State(sub_a, T=T_2, p=p_a2)
h_a2 = st_a2.h.to(SI.h)

st_v1 = State(sub_v, p=p_v1, T=T_1)
h_v1 = st_v1.h.to(SI.h)

# Use the previously defined st_v2
h_v2 = st_v2.h.to(SI.h)

st_liq = State(sub_v, p=p_v2, x=Q_(0.0, 'dimensionless'))
h_liq = st_liq.h.to(SI.h)

mdot_a = (-Qdot_out/(h_a2 - h_a1 + omega_2*h_v2 - omega_1*h_v1 + (omega_1 - omega_2)*h_liq)).to('kg/min')

v_a1 = st_a1.v.to(SI.v)
Vdot_1 = mdot_a*v_a1

p_a3 = p_3 - p_v3
st_a3 = State(sub_a, T=T_3, p=p_a3)
h_a3 = st_a3.h.to(SI.h)

st_v3 = State(sub_v, T=T_3, p=p_v3)
h_v3 = st_v3.h.to(SI.h)

Summarizing the states,

| State |           $h_a$            |           $h_v$            |          $h_{liq}$          |
|-------|----------------------------|----------------------------|-----------------------------|
|     1 | 427.46 kJ/kg | 2552.46 kJ/kg | ---                         |
|     2 | 408.93 kJ/kg | 2518.45 kJ/kg | 40.27 kJ/kg |
|     3 | 423.44 kJ/kg | 2545.58 kJ/kg | ---                         |

<div class="alert alert-success">
**Answer:** The mass flow rate of the air is $\dot{m}_a =$ 54.74 kg/min, the specific volume is $v_{a1} =$ 0.888 m<sup>3</sup>/kg, and the volumetric flow rate is $\dot{V}_1 =$ 48.59 m<sup>3</sup>/min
</div>

### 3. the mass flow rate of condensate

From the equations derived in the last part,

In [6]:
mdot_liq = mdot_a*(omega_1 - omega_2)

<div class="alert alert-success">

**Answer:** The mass flow rate of the condensate is $\dot{m}_{liq} =$ 0.51 kg/min

</div>

### 4. the rate of heat transfer in the heating unit

Drawing a control volume around the heating portion, with the constant composition process, we can find the energy balance as
$$\dot{Q}_{cv} = \dot{m}_a \left[h_{a3} - h_{a2} + \omega_2 \left(h_{v3} - h_{v2}\right)\right]$$

In [7]:
Qdot_in = (mdot_a*(h_a3 - h_a2 + omega_2*(h_v3 - h_v2))).to('kJ/min')

<div class="alert alert-success">

**Answer:** The rate of heat transfer into the moist air in the heating section is $\dot{Q}_{in} =$ 805.03 kJ/min

</div>

In [9]:
p_1 = Q_(0.99, 'atm').to('bar')  
print('p_1 = ',p_1)

Things you need 
V_1 into the system 
V_3 out of the system

Q_in across the Evaporator
Q_out across the Condesor


print('           ')
print('           ')
print('mdot_v1   =   {}'.format(m_v1))
print('mdot_v2   =   {}'.format(m_v2))
print('mdot_w    =   {}'.format(m_w))
print('mdot_a    =   {}'.format(m_a))
print('mdot_w2    =   {}'.format(m_a))
print('           ')
print ('T_1      = {}'.format(T_1))
print ('p_1      = {}'.format(p_1))
print ('pv_1     = {}'.format(pv_1))
print ('pa_1     = {}'.format(pa_1))
print ('vv_1     = {}'.format(vv_1))
print ('va_1     = {}'.format(va_1))
print('           ')
print ('ω_1      = {}'.format(ω_1))
print ('φ_1      = {}'.format(φ_1))
print('           ')
print ('T_2      = {}'.format(T_2))
print ('p_2      = {}'.format(p_2))
print ('pv_2     = {}'.format(pv_2))
print ('pa_2     = {}'.format(pa_2))
print ('vv_2     = {}'.format(vv_2))
print ('va_2     = {}'.format(va_2))
print('           ')
print ('ω_1      = {}'.format(ω_2))
print ('φ_1      = {}'.format(φ_2))
print('           ')
print ('T_3      = {}'.format(T_3))
print ('p_3      = {}'.format(p_3))
print ('pv_3     = {}'.format(pv_3))
print ('pa_3     = {}'.format(pa_3))
print ('vv_3     = {}'.format(vv_3))
print ('va_3     = {}'.format(va_3))
print('           ')
print ('ω_3      = {}'.format(ω_3))
print ('φ_3      = {}'.format(φ_3))
print('           ')
print ('T_w      = {}'.format(T_w)) #DewPoint



p_1 =  1.0031175 bar
